## Fetching data

In [62]:
import urllib.request
import os
import shutil

path='/mnt/All/Downloads/Datasets/spam'
paths=path+'/spam.tar.bz2'
pathh=path+'/ham.tar.bz2'
pathsd=path+'/spam/'
pathhd=path+'/easy_ham/'

url='https://spamassassin.apache.org/old/publiccorpus/'

def fetch_spam():
    if not os.path.isdir(pathsd):
        urllib.request.urlretrieve(url+'20030228_spam.tar.bz2',paths)
        shutil.unpack_archive(paths,path+'/')
        os.remove(paths)
    if not os.path.isdir(pathhd):
        urllib.request.urlretrieve(url+'20030228_easy_ham.tar.bz2',pathh)
        shutil.unpack_archive(pathh,path+'/')
        os.remove(pathh)

fetch_spam()

In [64]:
spam_files=[x for x in sorted(os.listdir(pathsd)) if len(x)>20]
ham_files=[x for x in sorted(os.listdir(pathhd)) if len(x)>20]

In [66]:
len(spam_files)

500

In [67]:
len(ham_files)

2500

In [105]:
import email.policy
import email

def open_mail(spam,file):
    if spam:
        dir=pathsd
    else:
        dir=pathhd 
    with open(dir+file,'rb') as mail:
        return email.parser.BytesParser(policy=email.policy.default).parse(mail)

In [98]:
ham=[open_mail(False,x) for x in ham_files]
spam=[open_mail(True,x) for x in spam_files]

In [119]:
print(spam[3].get_content().strip())

##################################################
#                                                #
#                 Adult Club                     #
#           Offers FREE Membership               #
#                                                #
##################################################

>>>>>  INSTANT ACCESS TO ALL SITES NOW
>>>>>  Your User Name And Password is.
>>>>>  User Name: zzzz@spamassassin.taint.org
>>>>>  Password: 760382

5 of the Best Adult Sites on the Internet for FREE!
---------------------------------------
NEWS 08/18/02
With just over 2.9 Million Members that signed up for FREE, Last month there were 721,184 New
Members. Are you one of them yet???
---------------------------------------
Our Membership FAQ

Q. Why are you offering free access to 5 adult membership sites for free?
A. I have advertisers that pay me for ad space so you don't have to pay for membership.

Q. Is it true my membership is for life?
A. Absolutely you'll never have to pay a cen

In [111]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [122]:
get_email_structure(spam[6])

'text/plain'

In [126]:
from collections import Counter

def count_types(list):
    structures = Counter()
    for email in list:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [130]:
count_types(spam)

Counter({'text/plain': 218,
         'text/html': 183,
         'multipart(text/plain, text/html)': 45,
         'multipart(text/html)': 20,
         'multipart(text/plain)': 19,
         'multipart(multipart(text/html))': 5,
         'multipart(text/plain, image/jpeg)': 3,
         'multipart(text/html, application/octet-stream)': 2,
         'multipart(text/plain, application/octet-stream)': 1,
         'multipart(text/html, text/plain)': 1,
         'multipart(multipart(text/html), application/octet-stream, image/jpeg)': 1,
         'multipart(multipart(text/plain, text/html), image/gif)': 1,
         'multipart/alternative': 1})

In [132]:
count_types(ham)

Counter({'text/plain': 2408,
         'multipart(text/plain, application/pgp-signature)': 66,
         'multipart(text/plain, text/html)': 8,
         'multipart(text/plain, text/plain)': 4,
         'multipart(text/plain)': 3,
         'multipart(text/plain, application/octet-stream)': 2,
         'multipart(text/plain, text/enriched)': 1,
         'multipart(text/plain, application/ms-tnef, text/plain)': 1,
         'multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)': 1,
         'multipart(text/plain, video/mng)': 1,
         'multipart(text/plain, multipart(text/plain))': 1,
         'multipart(text/plain, application/x-pkcs7-signature)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))': 1,
         'multipart(text/plain, application/x-java-applet)': 1})

In [136]:
for header, con in spam[0].items():
    print(header, ':', con)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu